# SMA+EMA+BB+ATR Strategy

In [100]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from datetime import time
import pytz


# PARAMETERS
symbol = "ITC.NS"
capital = 100000         # ₹1,00,000 starting capital
leverage = 5
atr_period = 14
atr_mult = 1.5            # ATR multiplier for stop loss
intraday_start = time(9, 15)
intraday_end = time(15, 30)
force_exit_time = time(15, 0)
ist = pytz.timezone('Asia/Kolkata')


# Download 5-minute data (~60 days)
data = yf.download(symbol, interval='5m', period='60d', progress=False, prepost=False, auto_adjust=True, multi_level_index=False)
data = data.tz_convert('Asia/Kolkata')
data = data.between_time(intraday_start, intraday_end).dropna()


# Calculate indicators
data['SMA5'] = talib.SMA(data['Close'], timeperiod=5)
data['EMA9'] = talib.EMA(data['Close'], timeperiod=9)
bb_upper, bb_middle, bb_lower = talib.BBANDS(data['Close'], timeperiod=20, nbdevup=2, nbdevdn=2)
data['BB_UPPER'] = bb_upper
data['BB_LOWER'] = bb_lower
data['ATR'] = talib.ATR(data['High'], data['Low'], data['Close'], timeperiod=atr_period)

# Helper functions for brokerage/costs
def brokerage(order_value):
    raw_brokerage = min(20, order_value * 0.001)
    if raw_brokerage < 5:
        return min(5, order_value * 0.025)
    return raw_brokerage


def stt(trade_val): return trade_val * 0.00025   # 0.025% on sell side only
def stamp_duty(trade_val): return trade_val * 0.00003   # 0.003% on buy side only
def exchange_charges(trade_val): return trade_val * 0.0000297   # 0.00297% both sides (NSE rate)
def sebi_charges(trade_val): return trade_val * 0.000001   # 0.0001% both sides
def ipf_charges(trade_val): return trade_val * 0.000001   # 0.0001% both sides
def gst(broker, exch, sebi): return 0.18 * (broker + exch + sebi)


# Trade simulation state
trades = []
position = None
stop_loss = None
max_price = None
min_price = None


for i in range(1, len(data)):
    row = data.iloc[i]
    prev = data.iloc[i-1]
    ts = row.name


    # Force exit before 15:00 IST
    if ts.time() >= force_exit_time:
        if position:
            exit_price = row['Close']
            exit_time_pos = ts
            qty = position['qty']

            entry_val = position['entry_price'] * qty
            exit_val = exit_price * qty

            # PnL calculation
            if position['direction'] == 'long':
                gross_pnl = (exit_price - position['entry_price']) * qty
                stt_ = 0                            # No STT on buy
                stamp = stamp_duty(entry_val)      # Stamp duty on buy side (entry)
            else:  # short
                gross_pnl = (position['entry_price'] - exit_price) * qty
                stt_ = stt(entry_val)              # STT on sell side (entry)
                stamp = stamp_duty(exit_val)       # Stamp duty on buy side (exit)

            # Charges
            broker = brokerage(entry_val) + brokerage(exit_val)
            exch = exchange_charges(entry_val) + exchange_charges(exit_val)
            sebi_ = sebi_charges(entry_val) + sebi_charges(exit_val)
            ipf_ = ipf_charges(entry_val) + ipf_charges(exit_val)
            gst_ = gst(broker, exch, sebi_)
            total_cost = broker + exch + sebi_ + ipf_ + stt_ + stamp + gst_

            net_pnl = gross_pnl - total_cost

            trades.append({
                'entry_time': position['entry_time'],
                'exit_time': exit_time_pos,
                'direction': position['direction'],
                'entry_price': position['entry_price'],
                'exit_price': exit_price,
                'qty': qty,
                'gross_pnl': gross_pnl,
                'net_pnl': net_pnl,
                'costs': total_cost,
            
                'brokerage': broker,
                'exchange_charges': exch,
                'sebi_charges': sebi_,
                'ipf_charges': ipf_,
                'stt': stt_,
                'stamp': stamp,
                'gst': gst_
            })
            position = None
        continue


    # Only consider bars with valid ATR and indicators
    if pd.isna(row['ATR']) or pd.isna(row['SMA5']) or pd.isna(row['EMA9']) or pd.isna(row['BB_UPPER']) or pd.isna(row['BB_LOWER']):
        continue


    # Entry signals
    long_signal = (prev['Close'] < prev['SMA5'] and prev['Close'] < prev['EMA9'] and
                   row['Close'] > row['SMA5'] and row['Close'] > row['EMA9'] and
                   row['Close'] > row['BB_LOWER'])
    short_signal = (prev['Close'] > prev['SMA5'] and prev['Close'] > prev['EMA9'] and
                    row['Close'] < row['SMA5'] and row['Close'] < row['EMA9'] and
                    row['Close'] < row['BB_UPPER'])


    price = row['Close']


    if position is None:
        max_cap = capital * leverage
        qty = int(max_cap // price)
        if qty == 0:
            continue
        if long_signal:
            position = {'entry_time': ts, 'entry_price': price, 'qty': qty, 'direction': 'long'}
            stop_loss = price - atr_mult * row['ATR']  # ATR-based stop loss
            max_price = price
        elif short_signal:
            position = {'entry_time': ts, 'entry_price': price, 'qty': qty, 'direction': 'short'}
            stop_loss = price + atr_mult * row['ATR']
            min_price = price


    elif position['direction'] == 'long':
        max_price = max(max_price, price)
        # Trail stop loss: highest price - ATR * multiplier
        new_stop = max_price - atr_mult * row['ATR']
        stop_loss = max(stop_loss, new_stop)
        if price <= stop_loss or ts.time() >= force_exit_time:
            exit_price = price
            exit_time_pos = ts
            qty = position['qty']

            entry_val = position['entry_price'] * qty
            exit_val = exit_price * qty

            gross_pnl = (exit_price - position['entry_price']) * qty
            stt_ = 0
            stamp = stamp_duty(entry_val)

            broker = brokerage(entry_val) + brokerage(exit_val)
            exch = exchange_charges(entry_val) + exchange_charges(exit_val)
            sebi_ = sebi_charges(entry_val) + sebi_charges(exit_val)
            ipf_ = ipf_charges(entry_val) + ipf_charges(exit_val)
            gst_ = gst(broker, exch, sebi_)

            total_cost = broker + exch + sebi_ + ipf_ + stt_ + stamp + gst_

            net_pnl = gross_pnl - total_cost

            trades.append({
                'entry_time': position['entry_time'],
                'exit_time': exit_time_pos,
                'direction': 'long',
                'entry_price': position['entry_price'],
                'exit_price': exit_price,
                'qty': qty,
                'gross_pnl': gross_pnl,
                'net_pnl': net_pnl,
                'costs': total_cost,

                'brokerage': broker,
                'exchange_charges': exch,
                'sebi_charges': sebi_,
                'ipf_charges': ipf_,
                'stt': stt_,
                'stamp': stamp,
                'gst': gst_
            })
            position = None


    elif position['direction'] == 'short':
        min_price = min(min_price, price)
        # Trail stop loss: lowest price + ATR * multiplier
        new_stop = min_price + atr_mult * row['ATR']
        stop_loss = min(stop_loss, new_stop)
        if price >= stop_loss or ts.time() >= force_exit_time:
            exit_price = price
            exit_time_pos = ts
            qty = position['qty']

            entry_val = position['entry_price'] * qty
            exit_val = price * qty

            gross_pnl = (position['entry_price'] - exit_price) * qty
            stt_ = stt(entry_val)
            stamp = stamp_duty(exit_val)

            broker = brokerage(entry_val) + brokerage(exit_val)
            exch = exchange_charges(entry_val) + exchange_charges(exit_val)
            sebi_ = sebi_charges(entry_val) + sebi_charges(exit_val)
            ipf_ = ipf_charges(entry_val) + ipf_charges(exit_val)
            gst_ = gst(broker, exch, sebi_)

            total_cost = broker + exch + sebi_ + ipf_ + stt_ + stamp + gst_

            net_pnl = gross_pnl - total_cost

            trades.append({
                'entry_time': position['entry_time'],
                'exit_time': exit_time_pos,
                'direction': 'short',
                'entry_price': position['entry_price'],
                'exit_price': exit_price,
                'qty': qty,
                'gross_pnl': gross_pnl,
                'net_pnl': net_pnl,
                'costs': total_cost,

                'brokerage': broker,
                'exchange_charges': exch,
                'sebi_charges': sebi_,
                'ipf_charges': ipf_,
                'stt': stt_,
                'stamp': stamp,
                'gst': gst_
            })
            position = None


# Export trades to CSV
trades_df = pd.DataFrame(trades)
trades_df.to_csv('intraday_scalping_trades_atr_stop.csv', index=False)


# Performance metrics
net_pnl = trades_df['net_pnl'].sum()
win_trades = (trades_df['net_pnl'] > 0).sum()
total_trades = len(trades_df)
win_rate = (win_trades / total_trades) * 100 if total_trades > 0 else 0
avg_trade_pnl = trades_df['net_pnl'].mean() if total_trades > 0 else 0
cum_pnl = trades_df['net_pnl'].cumsum()
max_drawdown = (cum_pnl.cummax() - cum_pnl).max() if total_trades > 0 else 0
total_costs = trades_df['costs'].sum()


print("Final Performance Metrics:")
print(f"Net PnL: ₹{net_pnl:,.2f}")
print(f"Win rate: {win_rate:.2f}%")
print(f"Avg trade PnL: ₹{avg_trade_pnl:,.2f}")
print(f"Max drawdown: ₹{max_drawdown:,.2f}")
print(f"Number of trades: {total_trades}")
print(f"Total brokerage/charges paid: ₹{total_costs:,.2f}")
print("Trade details exported to 'intraday_scalping_trades_atr_stop.csv'")


Final Performance Metrics:
Net PnL: ₹-6,429.79
Win rate: 40.56%
Avg trade PnL: ₹-35.72
Max drawdown: ₹19,509.09
Number of trades: 180
Total brokerage/charges paid: ₹27,887.91
Trade details exported to 'intraday_scalping_trades_atr_stop.csv'


# SMA+EMA+BB+ATR+RSI+MACD

In [93]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from datetime import time
import pytz


# PARAMETERS
symbol = "NIFTYBEES.NS"
capital = 100000           # ₹1,00,000 starting capital
leverage = 5
atr_period = 14
atr_mult = 1.5            # ATR multiplier for stop loss
intraday_start = time(9, 15)
intraday_end = time(15, 30)
force_exit_time = time(15, 0)
ist = pytz.timezone('Asia/Kolkata')


# Download 5-minute data (~60 days)
data = yf.download(symbol, interval='5m', period='60d', progress=False, prepost=False, auto_adjust=True, multi_level_index=False)
data = data.tz_convert('Asia/Kolkata')
data = data.between_time(intraday_start, intraday_end).dropna()


# Calculate indicators
data['SMA5'] = talib.SMA(data['Close'], timeperiod=5)
data['EMA9'] = talib.EMA(data['Close'], timeperiod=9)
bb_upper, bb_middle, bb_lower = talib.BBANDS(data['Close'], timeperiod=20, nbdevup=2, nbdevdn=2)
data['BB_UPPER'] = bb_upper
data['BB_LOWER'] = bb_lower
data['ATR'] = talib.ATR(data['High'], data['Low'], data['Close'], timeperiod=atr_period)

# Additional indicators: RSI and MACD
data['RSI'] = talib.RSI(data['Close'], timeperiod=14)
macd, macdsignal, macdhist = talib.MACD(data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
data['MACD'] = macd
data['MACD_signal'] = macdsignal


# Helper functions for brokerage/costs
def brokerage(order_value):
    raw_brokerage = min(20, order_value * 0.001)
    if raw_brokerage < 5:
        return min(5, order_value * 0.025)
    return raw_brokerage


def stt(trade_val): return trade_val * 0.00025   # 0.025% on sell side only
def stamp_duty(trade_val): return trade_val * 0.00003   # 0.003% on buy side only
def exchange_charges(trade_val): return trade_val * 0.0000297   # 0.00297% both sides (NSE rate)
def sebi_charges(trade_val): return trade_val * 0.000001   # 0.0001% both sides
def ipf_charges(trade_val): return trade_val * 0.000001   # 0.0001% both sides
def gst(broker, exch, sebi): return 0.18 * (broker + exch + sebi)


# Trade simulation state
trades = []
position = None
stop_loss = None
max_price = None
min_price = None


for i in range(1, len(data)):
    row = data.iloc[i]
    prev = data.iloc[i-1]
    ts = row.name

    # Force exit before 15:00 IST
    if ts.time() >= force_exit_time:
        if position:
            exit_price = row['Close']
            exit_time_pos = ts
            qty = position['qty']

            entry_val = position['entry_price'] * qty
            exit_val = exit_price * qty

            # PnL calculation
            if position['direction'] == 'long':
                gross_pnl = (exit_price - position['entry_price']) * qty
                stt_ = 0                            # No STT on buy
                stamp = stamp_duty(entry_val)      # Stamp duty on buy side (entry)
            else:  # short
                gross_pnl = (position['entry_price'] - exit_price) * qty
                stt_ = stt(entry_val)              # STT on sell side (entry)
                stamp = stamp_duty(exit_val)       # Stamp duty on buy side (exit)

            # Charges
            broker = brokerage(entry_val) + brokerage(exit_val)
            exch = exchange_charges(entry_val) + exchange_charges(exit_val)
            sebi_ = sebi_charges(entry_val) + sebi_charges(exit_val)
            ipf_ = ipf_charges(entry_val) + ipf_charges(exit_val)
            gst_ = gst(broker, exch, sebi_)
            total_cost = broker + exch + sebi_ + ipf_ + stt_ + stamp + gst_

            net_pnl = gross_pnl - total_cost

            trades.append({
                'entry_time': position['entry_time'],
                'exit_time': exit_time_pos,
                'direction': position['direction'],
                'entry_price': position['entry_price'],
                'exit_price': exit_price,
                'qty': qty,
                'gross_pnl': gross_pnl,
                'net_pnl': net_pnl,
                'total_cost': total_cost,
                'brokerage': broker,
                'stt': stt_,
                'stamp_duty': stamp,
                'exchange': exch,
                'sebi': sebi_,
                'ipf': ipf_,
                'gst': gst_
            })
            position = None
        continue


    # Skip bars without full indicator data
    if pd.isna(row['ATR']) or pd.isna(row['SMA5']) or pd.isna(row['EMA9']) or pd.isna(row['BB_UPPER']) or pd.isna(row['BB_LOWER']) or pd.isna(row['RSI']) or pd.isna(row['MACD']) or pd.isna(row['MACD_signal']):
        continue


    # Entry signals with additional RSI and MACD filters
    long_signal = (
        prev['Close'] < prev['SMA5']
        and prev['Close'] < prev['EMA9']
        and row['Close'] > row['SMA5']
        and row['Close'] > row['EMA9']
        and row['Close'] > row['BB_LOWER']
        and 40 < row['RSI'] < 80
        and row['MACD'] > row['MACD_signal']
    )
    short_signal = (
        prev['Close'] > prev['SMA5']
        and prev['Close'] > prev['EMA9']
        and row['Close'] < row['SMA5']
        and row['Close'] < row['EMA9']
        and row['Close'] < row['BB_UPPER']
        and 20 < row['RSI'] < 60
        and row['MACD'] < row['MACD_signal']
    )


    price = row['Close']


    if position is None:
        max_cap = capital * leverage
        qty = int(max_cap // price)
        if qty == 0:
            continue
        if long_signal:
            position = {'entry_time': ts, 'entry_price': price, 'qty': qty, 'direction': 'long'}
            stop_loss = price - atr_mult * row['ATR']  # ATR-based stop loss
            max_price = price
        elif short_signal:
            position = {'entry_time': ts, 'entry_price': price, 'qty': qty, 'direction': 'short'}
            stop_loss = price + atr_mult * row['ATR']
            min_price = price


    elif position['direction'] == 'long':
        max_price = max(max_price, price)
        # Trail stop loss: highest price - ATR * multiplier
        new_stop = max_price - atr_mult * row['ATR']
        stop_loss = max(stop_loss, new_stop)
        if price <= stop_loss or ts.time() >= force_exit_time:
            exit_price = price
            exit_time_pos = ts
            qty = position['qty']

            entry_val = position['entry_price'] * qty
            exit_val = exit_price * qty

            gross_pnl = (exit_price - position['entry_price']) * qty
            stt_ = 0
            stamp = stamp_duty(entry_val)

            broker = brokerage(entry_val) + brokerage(exit_val)
            exch = exchange_charges(entry_val) + exchange_charges(exit_val)
            sebi_ = sebi_charges(entry_val) + sebi_charges(exit_val)
            ipf_ = ipf_charges(entry_val) + ipf_charges(exit_val)
            gst_ = gst(broker, exch, sebi_)

            total_cost = broker + exch + sebi_ + ipf_ + stt_ + stamp + gst_

            net_pnl = gross_pnl - total_cost

            trades.append({
                'entry_time': position['entry_time'],
                'exit_time': exit_time_pos,
                'direction': 'long',
                'entry_price': position['entry_price'],
                'exit_price': exit_price,
                'qty': qty,
                'gross_pnl': gross_pnl,
                'net_pnl': net_pnl,
                'total_cost': total_cost,
                'brokerage': broker,
                'stt': stt_,
                'stamp_duty': stamp,
                'exchange': exch,
                'sebi': sebi_,
                'ipf': ipf_,
                'gst': gst_
            })
            position = None


    elif position['direction'] == 'short':
        min_price = min(min_price, price)
        # Trail stop loss: lowest price + ATR * multiplier
        new_stop = min_price + atr_mult * row['ATR']
        stop_loss = min(stop_loss, new_stop)
        if price >= stop_loss or ts.time() >= force_exit_time:
            exit_price = price
            exit_time_pos = ts
            qty = position['qty']

            entry_val = position['entry_price'] * qty
            exit_val = price * qty

            gross_pnl = (position['entry_price'] - exit_price) * qty
            stt_ = stt(entry_val)
            stamp = stamp_duty(exit_val)

            broker = brokerage(entry_val) + brokerage(exit_val)
            exch = exchange_charges(entry_val) + exchange_charges(exit_val)
            sebi_ = sebi_charges(entry_val) + sebi_charges(exit_val)
            ipf_ = ipf_charges(entry_val) + ipf_charges(exit_val)
            gst_ = gst(broker, exch, sebi_)

            total_cost = broker + exch + sebi_ + ipf_ + stt_ + stamp + gst_

            net_pnl = gross_pnl - total_cost

            trades.append({
                'entry_time': position['entry_time'],
                'exit_time': exit_time_pos,
                'direction': 'short',
                'entry_price': position['entry_price'],
                'exit_price': exit_price,
                'qty': qty,
                'gross_pnl': gross_pnl,
                'net_pnl': net_pnl,
                'total_cost': total_cost,
                'brokerage': broker,
                'stt': stt_,
                'stamp_duty': stamp,
                'exchange': exch,
                'sebi': sebi_,
                'ipf': ipf_,
                'gst': gst_
            })
            position = None


# Export trades to CSV
trades_df = pd.DataFrame(trades)
trades_df.to_csv('intraday_scalping_trades_atr_stop_with_filters.csv', index=False)


# Performance metrics
net_pnl = trades_df['net_pnl'].sum()
win_trades = (trades_df['net_pnl'] > 0).sum()
total_trades = len(trades_df)
win_rate = (win_trades / total_trades) * 100 if total_trades > 0 else 0
avg_trade_pnl = trades_df['net_pnl'].mean() if total_trades > 0 else 0
cum_pnl = trades_df['net_pnl'].cumsum()
max_drawdown = (cum_pnl.cummax() - cum_pnl).max() if total_trades > 0 else 0
total_costs = trades_df['total_cost'].sum()


print("Final Performance Metrics:")
print(f"Net PnL: ₹{net_pnl:,.2f}")
print(f"Win rate: {win_rate:.2f}%")
print(f"Avg trade PnL: ₹{avg_trade_pnl:,.2f}")
print(f"Max drawdown: ₹{max_drawdown:,.2f}")
print(f"Number of trades: {total_trades}")
print(f"Total brokerage/charges paid: ₹{total_costs:,.2f}")
print("Trade details exported to 'intraday_scalping_trades_atr_stop_with_filters.csv'")


Final Performance Metrics:
Net PnL: ₹-23,815.06
Win rate: 29.68%
Avg trade PnL: ₹-153.65
Max drawdown: ₹31,168.95
Number of trades: 155
Total brokerage/charges paid: ₹24,531.97
Trade details exported to 'intraday_scalping_trades_atr_stop_with_filters.csv'


# SMA+EMA+BB+ATR+RSI+MACD+VWAP

In [90]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from datetime import time
import pytz

# PARAMETERS
symbol = "NIFTYBEES.NS"
capital = 100_000
leverage = 5
atr_period = 14
atr_mult = 2.0
intraday_start = time(9, 15)
intraday_end = time(15, 30)
force_exit_time = time(15, 0)
ist = pytz.timezone("Asia/Kolkata")

# Thresholds
rsi_low, rsi_high = 40, 80
mfi_low, mfi_high = None, None  # not used

# COST MODEL
def brokerage(v):
    rb = min(20, v * 0.001)
    return min(5, v * 0.025) if rb < 5 else rb
def stt(v):    return v * 0.00025
def stamp(v):  return v * 0.00003
def exch(v):   return v * 0.0000297
def sebi(v):   return v * 0.000001
def ipf(v):    return v * 0.000001
def gst(b,e,s): return 0.18 * (b + e + s)

# FETCH DATA
df = yf.download(symbol, interval="5m", period="60d", progress=False, auto_adjust=True, multi_level_index=False)
df = df.tz_convert("Asia/Kolkata")
df = df.between_time(intraday_start, intraday_end).dropna()

# INDICATORS
df["SMA5"] = talib.SMA(df["Close"], 5)
df["EMA9"] = talib.EMA(df["Close"], 9)
bb_u, _, bb_l = talib.BBANDS(df["Close"], 20, 2, 2)
df["BB_UPPER"], df["BB_LOWER"] = bb_u, bb_l
df["ATR"] = talib.ATR(df["High"], df["Low"], df["Close"], atr_period)
df["RSI"] = talib.RSI(df["Close"], 14)
macd, macd_sig, _ = talib.MACD(df["Close"], 12, 26, 9)
df["MACD"], df["MACD_signal"] = macd, macd_sig

# VWAP
typ = (df["High"] + df["Low"] + df["Close"]) / 3
cum_vol = df["Volume"].cumsum()
cum_pv  = (typ * df["Volume"]).cumsum()
df["VWAP"] = cum_pv / cum_vol

# SIMULATE TRADES
trades = []
pos = None

for i in range(1, len(df)):
    row, prev = df.iloc[i], df.iloc[i-1]
    t = row.name.time()

    # Force exit at/after 15:00
    if pos and t >= force_exit_time:
        px, tx = row["Close"], row.name
        q = pos["qty"]
        ev, xv = pos["entry_price"]*q, px*q
        gross = (px-pos["entry_price"])*q if pos["dir"]=="long" else (pos["entry_price"]-px)*q
        sttax = 0 if pos["dir"]=="long" else stt(ev)
        stduty= stamp(ev) if pos["dir"]=="long" else stamp(xv)
        brk = brokerage(ev)+brokerage(xv)
        exc = exch(ev)+exch(xv)
        sb  = sebi(ev)+sebi(xv)
        ip  = ipf(ev)+ipf(xv)
        gstx= gst(brk, exc, sb)
        cost= brk+exc+sb+ip+sttax+stduty+gstx
        net = gross - cost
        trades.append({"entry_time":pos["entry_time"],"exit_time":tx,
                       "dir":pos["dir"],"entry":pos["entry_price"],
                       "exit":px,"qty":q,"gross":gross,"net":net,
                       "brokerage":brk,"stt":sttax,"stamp":stduty,
                       "exchange":exc,"sebi":sb,"ipf":ip,"gst":gstx})
        pos = None

    # Skip if any required indicator is NaN
    req = ["SMA5","EMA9","BB_UPPER","BB_LOWER","ATR","RSI","MACD","MACD_signal","VWAP"]
    if row[req].isna().any(): continue

    # Entry signals
    long_sig = (
        prev["Close"]<prev["SMA5"] and prev["Close"]<prev["EMA9"] and
        row["Close"]>row["SMA5"] and row["Close"]>row["EMA9"] and
        row["Close"]>row["BB_LOWER"] and
        rsi_low<row["RSI"]<rsi_high and
        row["MACD"]>row["MACD_signal"] and
        row["Close"]>row["VWAP"]
    )
    short_sig = (
        prev["Close"]>prev["SMA5"] and prev["Close"]>prev["EMA9"] and
        row["Close"]<row["SMA5"] and row["Close"]<row["EMA9"] and
        row["Close"]<row["BB_UPPER"] and
        rsi_low<row["RSI"]<rsi_high and
        row["MACD"]<row["MACD_signal"] and
        row["Close"]<row["VWAP"]
    )

    price = row["Close"]
    if not pos:
        maxv = capital * leverage
        q = int(maxv//price)
        if q and long_sig:
            pos = {"entry_time":row.name,"entry_price":price,"qty":q,"dir":"long"}
            sl = price - atr_mult*row["ATR"]; maxp = price
        elif q and short_sig:
            pos = {"entry_time":row.name,"entry_price":price,"qty":q,"dir":"short"}
            sl = price + atr_mult*row["ATR"]; minp = price
    else:
        # Trailing stop
        if pos["dir"]=="long":
            maxp = max(maxp, price)
            sl = max(sl, maxp - atr_mult*row["ATR"])
            if price <= sl:
                px, tx = price, row.name
                q=pos["qty"]; ev=pos["entry_price"]*q; xv=px*q
                gross=(px-pos["entry_price"])*q
                sttax, stduty = 0, stamp(ev)
                brk=brokerage(ev)+brokerage(xv)
                exc=exch(ev)+exch(xv); sb=sebi(ev)+sebi(xv)
                ip=ipf(ev)+ipf(xv); gstx=gst(brk,exc,sb)
                cost=brk+exc+sb+ip+sttax+stduty+gstx
                net=gross-cost
                trades.append({"entry_time":pos["entry_time"],"exit_time":tx,
                               "dir":"long","entry":pos["entry_price"],"exit":px,
                               "qty":q,"gross":gross,"net":net,
                               "brokerage":brk,"stt":sttax,"stamp":stduty,
                               "exchange":exc,"sebi":sb,"ipf":ip,"gst":gstx})
                pos=None
        else:
            minp=min(minp, price)
            sl=min(sl, minp + atr_mult*row["ATR"])
            if price >= sl:
                px, tx=price, row.name
                q=pos["qty"]; ev=pos["entry_price"]*q; xv=px*q
                gross=(pos["entry_price"]-px)*q
                sttax, stduty=stt(ev), stamp(xv)
                brk=brokerage(ev)+brokerage(xv)
                exc=exch(ev)+exch(xv); sb=sebi(ev)+sebi(xv)
                ip=ipf(ev)+ipf(xv); gstx=gst(brk,exc,sb)
                cost=brk+exc+sb+ip+sttax+stduty+gstx
                net=gross-cost
                trades.append({"entry_time":pos["entry_time"],"exit_time":tx,
                               "dir":"short","entry":pos["entry_price"],"exit":px,
                               "qty":q,"gross":gross,"net":net,
                               "brokerage":brk,"stt":sttax,"stamp":stduty,
                               "exchange":exc,"sebi":sb,"ipf":ip,"gst":gstx})
                pos=None

# OUTPUT
out = pd.DataFrame(trades)
out.to_csv("scalp_SMA_EMA_BB_ATR_RSI_MACD_VWAP.csv", index=False)

net = out["net"].sum()
wins = (out["net"]>0).sum(); n=len(out)
winr = wins/n*100 if n else 0
avgp= out["net"].mean() if n else 0
cum= out["net"].cumsum(); mdd=(cum.cummax()-cum).max() if n else 0
costs= out[["brokerage","stt","stamp","exchange","sebi","ipf","gst"]].sum().sum()

print(f"Net PnL: ₹{net:,.2f}")
print(f"Win rate: {winr:.2f}%")
print(f"Avg trade PnL: ₹{avgp:,.2f}")
print(f"Max drawdown: ₹{mdd:,.2f}")
print(f"Number of trades: {n}")
print(f"Total brokerage/charges paid: ₹{costs:,.2f}")


Net PnL: ₹-11,207.44
Win rate: 26.51%
Avg trade PnL: ₹-135.03
Max drawdown: ₹16,915.82
Number of trades: 83
Total brokerage/charges paid: ₹10,500.62


# SMA+EMA+BB+ATR+RSI+MACD+Stochastic

In [89]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from datetime import time
import pytz

# PARAMETERS
symbol = "POWERGRID.NS"
capital = 100_000
leverage = 5
atr_period = 14
atr_mult = 2.0
intraday_start = time(9, 15)
intraday_end = time(15, 30)
force_exit_time = time(15, 0)
ist = pytz.timezone("Asia/Kolkata")

# RSI & Stochastic thresholds
rsi_low, rsi_high = 40, 80
mfi_low, mfi_high = None, None  # Not used
sto_k, sto_d = 5, 3

# COST MODEL
def brokerage(v):
    rb = min(20, v * 0.001)
    return min(5, v * 0.025) if rb < 5 else rb
def stt(v):    return v * 0.00025
def stamp(v):  return v * 0.00003
def exch(v):   return v * 0.0000297
def sebi(v):   return v * 0.000001
def ipf(v):    return v * 0.000001
def gst(b,e,s): return 0.18 * (b + e + s)

# FETCH AND PREPARE DATA
df = yf.download(symbol, interval="5m", period="60d", progress=False, auto_adjust=True, multi_level_index=False)
df = df.tz_convert("Asia/Kolkata")
df = df.between_time(intraday_start, intraday_end).dropna()

# INDICATORS
df["SMA5"] = talib.SMA(df["Close"], 5)
df["EMA9"] = talib.EMA(df["Close"], 9)
bb_u, _, bb_l = talib.BBANDS(df["Close"], 20, 2, 2)
df["BB_UPPER"], df["BB_LOWER"] = bb_u, bb_l
df["ATR"] = talib.ATR(df["High"], df["Low"], df["Close"], atr_period)
df["RSI"] = talib.RSI(df["Close"], 14)
macd, macd_sig, _ = talib.MACD(df["Close"], 12, 26, 9)
df["MACD"], df["MACD_signal"] = macd, macd_sig
sto_k_vals, sto_d_vals = talib.STOCH(
    df["High"], df["Low"], df["Close"],
    fastk_period=sto_k, slowk_period=sto_d,
    slowd_period=sto_d
)
df["STOCHK"], df["STOCHD"] = sto_k_vals, sto_d_vals

# TRADE SIMULATION
trades = []
position = None

for i in range(1, len(df)):
    row, prev = df.iloc[i], df.iloc[i-1]
    t = row.name.time()

    # Forced exit at or after 15:00 IST
    if position and t >= force_exit_time:
        px, tx = row["Close"], row.name
        q = position["qty"]
        ev, xv = position["entry_price"]*q, px*q
        gross = (px - position["entry_price"])*q if position["dir"]=="long" else (position["entry_price"] - px)*q
        sttax = 0 if position["dir"]=="long" else stt(ev)
        stduty = stamp(ev) if position["dir"]=="long" else stamp(xv)
        brk = brokerage(ev)+brokerage(xv)
        excv = exch(ev)+exch(xv)
        sb = sebi(ev)+sebi(xv)
        ip = ipf(ev)+ipf(xv)
        gstx = gst(brk, excv, sb)
        cost = brk+excv+sb+ip+sttax+stduty+gstx
        net = gross - cost
        trades.append({
            "entry_time": position["entry_time"], "exit_time": tx,
            "dir": position["dir"], "entry": position["entry_price"],
            "exit": px, "qty": q, "gross": gross, "net": net,
            "brokerage": brk, "stt": sttax, "stamp": stduty,
            "exchange": excv, "sebi": sb, "ipf": ip, "gst": gstx
        })
        position = None

    # Skip if any indicator is NaN
    if row[["SMA5","EMA9","BB_UPPER","BB_LOWER","ATR","RSI","MACD","MACD_signal","STOCHK","STOCHD"]].isna().any():
        continue

    # ENTRY SIGNALS
    long_sig = (
        prev["Close"]<prev["SMA5"] and prev["Close"]<prev["EMA9"] and
        row["Close"]>row["SMA5"] and row["Close"]>row["EMA9"] and
        row["Close"]>row["BB_LOWER"] and
        rsi_low<row["RSI"]<rsi_high and
        row["MACD"]>row["MACD_signal"] and
        row["STOCHK"]<20 and row["STOCHK"]>row["STOCHD"]
    )
    short_sig = (
        prev["Close"]>prev["SMA5"] and prev["Close"]>prev["EMA9"] and
        row["Close"]<row["SMA5"] and row["Close"]<row["EMA9"] and
        row["Close"]<row["BB_UPPER"] and
        rsi_low<row["RSI"]<rsi_high and
        row["MACD"]<row["MACD_signal"] and
        row["STOCHK"]>80 and row["STOCHK"]<row["STOCHD"]
    )

    price = row["Close"]
    if not position:
        max_val = capital * leverage
        qty = int(max_val // price)
        if qty and long_sig:
            position = {"entry_time": row.name, "entry_price": price, "qty": qty, "dir": "long"}
            sl = price - atr_mult * row["ATR"]
            maxp = price
        elif qty and short_sig:
            position = {"entry_time": row.name, "entry_price": price, "qty": qty, "dir": "short"}
            sl = price + atr_mult * row["ATR"]
            minp = price
    else:
        # TRAILING STOP
        if position["dir"]=="long":
            maxp = max(maxp, price)
            sl = max(sl, maxp - atr_mult * row["ATR"])
            if price <= sl:
                px, tx = price, row.name
                q=position["qty"]; ev=position["entry_price"]*q; xv=px*q
                gross=(px-position["entry_price"])*q
                sttax, stduty=0, stamp(ev)
                brk=brokerage(ev)+brokerage(xv)
                excv=exch(ev)+exch(xv); sb=sebi(ev)+sebi(xv)
                ip=ipf(ev)+ipf(xv); gstx=gst(brk,excv,sb)
                cost=brk+excv+sb+ip+sttax+stduty+gstx
                net=gross-cost
                trades.append({"entry_time":position["entry_time"],"exit_time":tx,
                               "dir":"long","entry":position["entry_price"],"exit":px,
                               "qty":q,"gross":gross,"net":net,
                               "brokerage":brk,"stt":sttax,"stamp":stduty,
                               "exchange":excv,"sebi":sb,"ipf":ip,"gst":gstx})
                position=None
        else:
            minp=min(minp, price)
            sl=min(sl, minp + atr_mult*row["ATR"])
            if price >= sl:
                px, tx=price, row.name
                q=position["qty"]; ev=position["entry_price"]*q; xv=px*q
                gross=(position["entry_price"]-px)*q
                sttax, stduty=stt(ev), stamp(xv)
                brk=brokerage(ev)+brokerage(xv)
                excv=exch(ev)+exch(xv); sb=sebi(ev)+sebi(xv)
                ip=ipf(ev)+ipf(xv); gstx=gst(brk,excv,sb)
                cost=brk+excv+sb+ip+sttax+stduty+gstx
                net=gross-cost
                trades.append({"entry_time":position["entry_time"],"exit_time":tx,
                               "dir":"short","entry":position["entry_price"],"exit":px,
                               "qty":q,"gross":gross,"net":net,
                               "brokerage":brk,"stt":sttax,"stamp":stduty,
                               "exchange":excv,"sebi":sb,"ipf":ip,"gst":gstx})
                position=None

# EXPORT & METRICS
out = pd.DataFrame(trades)
out.to_csv("scalp_SMA_EMA_BB_ATR_RSI_MACD_STOCH.csv", index=False)

net = out["net"].sum()
wins = (out["net"]>0).sum(); n = len(out)
win_rate = wins/n*100 if n else 0
avg_pnl = out["net"].mean() if n else 0
cum = out["net"].cumsum(); mdd = (cum.cummax()-cum).max() if n else 0
costs = out[["brokerage","stt","stamp","exchange","sebi","ipf","gst"]].sum().sum()

print(f"Net PnL: ₹{net:,.2f}")
print(f"Win rate: {win_rate:.2f}%")
print(f"Avg trade PnL: ₹{avg_pnl:,.2f}")
print(f"Max drawdown: ₹{mdd:,.2f}")
print(f"Number of trades: {n}")
print(f"Total brokerage/charges paid: ₹{costs:,.2f}")


Net PnL: ₹16,765.60
Win rate: 100.00%
Avg trade PnL: ₹5,588.53
Max drawdown: ₹0.00
Number of trades: 3
Total brokerage/charges paid: ₹298.87


# SMA+EMA+BB+ATR+RSI+MACD+MFI

In [84]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from datetime import time
import pytz

# PARAMETERS
symbol = "RELIANCE.NS"
capital = 100_000
leverage = 5
atr_period = 14
atr_mult = 2.0
intraday_start = time(9, 15)
intraday_end = time(15, 30)
force_exit_time = time(15, 0)
ist = pytz.timezone('Asia/Kolkata')

# Indicator thresholds
rsi_long_low, rsi_long_high = 40, 80
rsi_short_low, rsi_short_high = 20, 60
mfi_low, mfi_high = 20, 80  # MFI oversold/overbought

# Download data
data = yf.download(symbol, interval='5m', period='60d',
                   progress=False, auto_adjust=True, multi_level_index=False)
data = data.tz_convert('Asia/Kolkata')
data = data.between_time(intraday_start, intraday_end).dropna()

# Calculate indicators
data['SMA5'] = talib.SMA(data['Close'], timeperiod=5)
data['EMA9'] = talib.EMA(data['Close'], timeperiod=9)
bb_u, bb_m, bb_l = talib.BBANDS(data['Close'], timeperiod=20, nbdevup=2, nbdevdn=2)
data['BB_UPPER'], data['BB_LOWER'] = bb_u, bb_l
data['ATR'] = talib.ATR(data['High'], data['Low'], data['Close'], timeperiod=atr_period)
data['RSI'] = talib.RSI(data['Close'], timeperiod=14)
macd, macd_sig, _ = talib.MACD(data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
data['MACD'], data['MACD_signal'] = macd, macd_sig
data['MFI'] = talib.MFI(data['High'], data['Low'], data['Close'], data['Volume'], timeperiod=14)

# Cost model functions
def brokerage(v):
    rb = min(20, v * 0.001)
    return min(5, v * 0.025) if rb < 5 else rb

def stt(v):    return v * 0.00025
def stamp(v):  return v * 0.00003
def exch(v):   return v * 0.0000297
def sebi(v):   return v * 0.000001
def ipf(v):    return v * 0.000001
def gst(b,e,s): return 0.18 * (b + e + s)

# Simulate trades
trades = []
position = None

for i in range(1, len(data)):
    row, prev = data.iloc[i], data.iloc[i-1]
    ts = row.name

    # Forced exit at or after 15:00
    if ts.time() >= force_exit_time and position:
        exit_price, exit_time = row['Close'], ts
        qty = position['qty']
        ent_val = position['entry_price'] * qty
        ext_val = exit_price * qty
        if position['dir']=='long':
            gross = (exit_price - position['entry_price']) * qty
            sttax, stduty = 0, stamp(ent_val)
        else:
            gross = (position['entry_price'] - exit_price) * qty
            sttax, stduty = stt(ent_val), stamp(ext_val)
        brk = brokerage(ent_val) + brokerage(ext_val)
        exc = exch(ent_val) + exch(ext_val)
        sb  = sebi(ent_val) + sebi(ext_val)
        ip  = ipf(ent_val) + ipf(ext_val)
        gstx= gst(brk, exc, sb)
        total_cost = brk + exc + sb + ip + sttax + stduty + gstx
        net = gross - total_cost
        trades.append({
            'entry_time': position['entry_time'], 'exit_time': exit_time,
            'dir': position['dir'], 'entry': position['entry_price'],
            'exit': exit_price, 'qty': qty, 'gross': gross, 'net': net,
            'brokerage': brk, 'stt': sttax, 'stamp': stduty,
            'exchange': exc, 'sebi': sb, 'ipf': ip, 'gst': gstx
        })
        position = None
        continue

    # Skip if any indicator is NaN
    if row[['SMA5','EMA9','BB_UPPER','BB_LOWER','ATR','RSI','MACD','MACD_signal','MFI']].isna().any():
        continue

    # Entry conditions
    long_signal = (
        prev['Close'] < prev['SMA5'] and prev['Close'] < prev['EMA9'] and
        row['Close'] > row['SMA5'] and row['Close'] > row['EMA9'] and
        row['Close'] > row['BB_LOWER'] and
        rsi_long_low < row['RSI'] < rsi_long_high and
        row['MACD'] > row['MACD_signal'] and
        row['MFI'] < mfi_low
    )
    short_signal = (
        prev['Close'] > prev['SMA5'] and prev['Close'] > prev['EMA9'] and
        row['Close'] < row['SMA5'] and row['Close'] < row['EMA9'] and
        row['Close'] < row['BB_UPPER'] and
        rsi_short_low < row['RSI'] < rsi_short_high and
        row['MACD'] < row['MACD_signal'] and
        row['MFI'] > mfi_high
    )

    price = row['Close']
    if position is None:
        max_cap = capital * leverage
        qty = int(max_cap // price)
        if qty and long_signal:
            position = {'entry_time': ts, 'entry_price': price, 'qty': qty, 'dir': 'long'}
            sl = price - atr_mult * row['ATR']
            maxp = price
        elif qty and short_signal:
            position = {'entry_time': ts, 'entry_price': price, 'qty': qty, 'dir': 'short'}
            sl = price + atr_mult * row['ATR']
            minp = price
    else:
        # Trailing stop logic
        if position['dir']=='long':
            maxp = max(maxp, price)
            sl = max(sl, maxp - atr_mult * row['ATR'])
            if price <= sl:
                exit_price, exit_time = price, ts
                qty = position['qty']
                ent_val, ext_val = position['entry_price']*qty, exit_price*qty
                gross = (exit_price - position['entry_price']) * qty
                sttax, stduty = 0, stamp(ent_val)
                brk = brokerage(ent_val) + brokerage(ext_val)
                exc = exch(ent_val) + exch(ext_val)
                sb  = sebi(ent_val) + sebi(ext_val)
                ip  = ipf(ent_val) + ipf(ext_val)
                gstx= gst(brk, exc, sb)
                total_cost = brk + exc + sb + ip + sttax + stduty + gstx
                net = gross - total_cost
                trades.append({
                    'entry_time': position['entry_time'], 'exit_time': exit_time,
                    'dir':'long','entry':position['entry_price'],'exit':exit_price,
                    'qty':qty,'gross':gross,'net':net,'brokerage':brk,
                    'stt':sttax,'stamp':stduty,'exchange':exc,
                    'sebi':sb,'ipf':ip,'gst':gstx
                })
                position = None
        else:
            minp = min(minp, price)
            sl = min(sl, minp + atr_mult * row['ATR'])
            if price >= sl:
                exit_price, exit_time = price, ts
                qty = position['qty']
                ent_val, ext_val = position['entry_price']*qty, exit_price*qty
                gross = (position['entry_price'] - exit_price) * qty
                sttax, stduty = stt(ent_val), stamp(ext_val)
                brk = brokerage(ent_val) + brokerage(ext_val)
                exc = exch(ent_val) + exch(ext_val)
                sb  = sebi(ent_val) + sebi(ext_val)
                ip  = ipf(ent_val) + ipf(ext_val)
                gstx= gst(brk, exc, sb)
                total_cost = brk + exc + sb + ip + sttax + stduty + gstx
                net = gross - total_cost
                trades.append({
                    'entry_time': position['entry_time'], 'exit_time': exit_time,
                    'dir':'short','entry':position['entry_price'],'exit':exit_price,
                    'qty':qty,'gross':gross,'net':net,'brokerage':brk,
                    'stt':sttax,'stamp':stduty,'exchange':exc,
                    'sebi':sb,'ipf':ip,'gst':gstx
                })
                position = None

# Export and performance
df = pd.DataFrame(trades)
df.to_csv('scalping_sma_ema_bb_atr_rsi_macd_mfi.csv', index=False)

net = df['net'].sum()
wins = (df['net']>0).sum(); total = len(df)
winr = wins/total*100 if total else 0
avg = df['net'].mean() if total else 0
cum = df['net'].cumsum(); mdd = (cum.cummax()-cum).max() if total else 0
costs = df[['brokerage','stt','stamp','exchange','sebi','ipf','gst']].sum().sum()

print(f"Net PnL: ₹{net:,.2f}")
print(f"Win rate: {winr:.2f}%")
print(f"Avg trade PnL: ₹{avg:,.2f}")
print(f"Max drawdown: ₹{mdd:,.2f}")
print(f"Number of trades: {total}")
print(f"Total brokerage/charges paid: ₹{costs:,.2f}")
print("Detailed trades in 'scalping_sma_ema_bb_atr_rsi_macd_mfi.csv'")


Net PnL: ₹-894.05
Win rate: 0.00%
Avg trade PnL: ₹-894.05
Max drawdown: ₹0.00
Number of trades: 1
Total brokerage/charges paid: ₹224.05
Detailed trades in 'scalping_sma_ema_bb_atr_rsi_macd_mfi.csv'


# SMA+EMA+BB+ATR+RSI+MACD+ADX